## 🎉 [Day 24](https://adventofcode.com/2018/day/24)

In [0]:
"""Some display functions"""
"""Uses: https://github.com/carpedm20/emoji"""
"""pip install emoji"""

import emoji

types_to_emoji = {
    'radiation': ':radioactive:',
    'slashing': ':dizzy:',
    'cold': ':snowflake',
    'bludgeoning': ':oncoming_fist:',
    'fire': ':fire:'}

team_to_emoji = {
    'Infection': ':smiling_face_with_horns:',
    'Immune System': ':smiling_face_with_halo:'}

def display_group(group, full=True):
  if full:
    return emoji.emojize('[G%02d %s] [%d/%d units] %d HP - %d %s DMG - RECEIVE: (%s)' % (
        group.group_id, team_to_emoji[group.team], group.num_units, group.initial_num_units,
        group.hp, group.damage_value, types_to_emoji[group.damage_type],
        ', '.join('%s: %d' % (v, group.damage_taken[k]) for k, v in types_to_emoji.items())))
  else:
    return emoji.emojize('[G%02d %s] [%d/%d units]' % (
        group.group_id, team_to_emoji[group.team], group.num_units, group.initial_num_units))
  
def display_attack(group, target, damage, remove_units):
  print(emoji.emojize('[G%02d %s] :crossed_swords: (%d %s) (-%d units) [G%02d %s]' % (
      group.group_id, team_to_emoji[group.team], damage, types_to_emoji[group.damage_type], 
      remove_units, target.group_id, team_to_emoji[target.team])), end=', ')

def display_round(round_id, groups, full=True):
  print('\n\nRound %d' % round_id)
  print(('\n' if full else ', ').join(display_group(g, full=full) for g in groups if g.num_units > 0))

In [0]:
import collections

class Group:
  """Define a group... Feels like Day 15 again :-)"""
  
  def __init__(self, group_id, team, num_units, hp, damage_type, damage_value, initiative):
    """Init a group stats"""
    self.group_id = group_id           # for display
    self.initial_num_units = num_units # for display
    self.team = team
    self.num_units = num_units
    self.hp = hp
    self.damage_type = damage_type
    self.damage_value = damage_value
    self.initiative = initiative
    self.damage_taken = collections.defaultdict(lambda: 1)
    self.target = None
    self.is_targeted = False
    
  def set_weak(self, tp):
    """Add weakness to given type"""
    self.damage_taken[tp] = 2
    
  def set_immune(self, tp):
    """Add immunity to given type"""
    self.damage_taken[tp] = 0
    
  def effective_power(self):
    """Group's effective power"""
    return self.num_units * self.damage_value
  
  def projected_damage(self, group):
    """How much damage would the current group do to the given group"""
    return self.effective_power() * group.damage_taken[self.damage_type]
  
  def attacks(self, verbose=True):
    """Attacks current target"""
    if self.target is not None:
      self.target.is_targeted = False
      if self.num_units > 0:
        damage = self.projected_damage(self.target) 
        remove_units = damage // self.target.hp
        if verbose:
          display_attack(self, self.target, damage, remove_units)
        self.target.num_units -= remove_units
        if self.target.num_units <= 0:
          return self.target
    

def parse_inputs(inputs, immune_boost):
  """Parse groups from the input string"""
  groups = []
  teams = inputs.split('\n\n')
  for t in teams:
    lines = t.splitlines()
    team = lines[0].replace(':', '')
    boost = immune_boost if team == 'Immune System' else 0
    for group_id, line in enumerate(lines[1:]):
      # main stats
      aux = line.split()
      num_units, hp, damage_value, initiative = [int(x) for x in aux if x.isdigit()]
      damage_type = aux[-5]
      g = Group(group_id + 1, team, num_units, hp, damage_type, damage_value + boost, initiative)
      # immunities
      if '(' in line:
        for block in line.split('(')[1].split(')')[0].split(';'):
          category, _, types = block.strip().split(' ', 2)
          for tp in types.split(', '):
            getattr(g, 'set_%s' % category)(tp)
      groups.append(g)
  return groups
  
  
def play(inputs, num_rounds=-1, immune_boost=0, verbose=True):
  """Play the game for `num_rounds` if specified and add `immune_boost` damages to the immune system groups"""
  # Init
  n = 0
  groups = parse_inputs(inputs, immune_boost=immune_boost)
  if verbose:
    display_round(0, groups, full=True)  
  
  while 1:    
    # Clean groups
    groups = [g for g in groups if g.num_units > 0]
    if verbose:
      display_round(n + 1, groups, full=False)
    
    # Target selection 
    # Consider groups first by effective power, then by initative
    for g in sorted(groups, key=lambda g: (- g.effective_power(), - g.initiative)):
      g.target = None
      targets = [o for o in groups if o.team != g.team and not o.is_targeted]
      if len(targets) > 0:
        target = max(targets, key=lambda t: (g.projected_damage(t), t.effective_power(), t.initiative))
        if g.projected_damage(target) > 0:
          g.target = target
          target.is_targeted = True
    
    # Attacking
    # Consider groups first by effective power, then by initative
    for g in sorted(groups, key=lambda g: - g.initiative):
      target = g.attacks(verbose=verbose)
      ## End when team wiped out
      if target is not None:
        num_targets = sum([(x.team == target.team) and (x.num_units > 0) for x in groups])
        if num_targets == 0:
          display_round(n + 1, groups, full=True)
          print()
          print('Winning team:', g.team)
          print('Remaining units:', sum([max(0, x.num_units) for x in groups if x.team == g.team]))
          return
        
    ## End when `num_rounds` reached
    n += 1
    if num_rounds > 0 and n >= num_rounds:
      display_round(n + 1, groups, full=True)
      break

In [6]:
%%time
with open('day24.txt', 'r') as f:
  inputs = f.read()

play(inputs, num_rounds=-1., immune_boost=0, verbose=False)
play(inputs, num_rounds=-1., immune_boost=25, verbose=False)



Round 529
[G01 😈] [1910/1994 units] 48414 HP - 46 :snowflake DMG - RECEIVE: (☢: 1, 💫: 0, ❄ 1, 👊: 1, 🔥: 1)
[G03 😈] [2976/3050 units] 29546 HP - 19 🔥 DMG - RECEIVE: (☢: 1, 💫: 1, ❄ 1, 👊: 1, 🔥: 0)
[G05 😈] [37/37 units] 30072 HP - 1365 :snowflake DMG - RECEIVE: (☢: 1, 💫: 1, ❄ 1, 👊: 1, 🔥: 1)
[G06 😈] [171/189 units] 49726 HP - 514 💫 DMG - RECEIVE: (☢: 1, 💫: 1, ❄ 1, 👊: 2, 🔥: 1)
[G07 😈] [189/930 units] 39623 HP - 81 👊 DMG - RECEIVE: (☢: 2, 💫: 1, ❄ 1, 👊: 1, 🔥: 1)
[G08 😈] [6325/6343 units] 31638 HP - 9 👊 DMG - RECEIVE: (☢: 1, 💫: 0, ❄ 1, 👊: 1, 🔥: 1)
[G10 😈] [3191/3198 units] 25539 HP - 15 👊 DMG - RECEIVE: (☢: 0, 💫: 1, ❄ 1, 👊: 1, 🔥: 0)

Winning team: Infection
Remaining units: 14799


Round 3875
[G02 😇] [699/2698 units] 9598 HP - 54 💫 DMG - RECEIVE: (☢: 1, 💫: 0, ❄ 1, 👊: 1, 🔥: 1)
[G03 😇] [993/4682 units] 6161 HP - 38 ☢ DMG - RECEIVE: (☢: 1, 💫: 1, ❄ 1, 👊: 1, 🔥: 1)
[G05 😇] [432/582 units] 3649 HP - 71 💫 DMG - RECEIVE: (☢: 1, 💫: 1, ❄ 1, 👊: 1, 🔥: 1)
[G06 😇] [37/53 units] 5147 HP - 853 :snowflake DMG -

In [7]:
#@title Example visualization on the Toy Example
inputs = """Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4"""

play(inputs, num_rounds=-1, immune_boost=0)



Round 0
[G01 😇] [17/17 units] 5390 HP - 4507 🔥 DMG - RECEIVE: (☢: 2, 💫: 1, ❄ 1, 👊: 2, 🔥: 1)
[G02 😇] [989/989 units] 1274 HP - 25 💫 DMG - RECEIVE: (☢: 1, 💫: 2, ❄ 1, 👊: 2, 🔥: 0)
[G01 😈] [801/801 units] 4706 HP - 116 👊 DMG - RECEIVE: (☢: 2, 💫: 1, ❄ 1, 👊: 1, 🔥: 1)
[G02 😈] [4485/4485 units] 2961 HP - 12 💫 DMG - RECEIVE: (☢: 0, 💫: 1, ❄ 2, 👊: 1, 🔥: 2)


Round 1
[G01 😇] [17/17 units], [G02 😇] [989/989 units], [G01 😈] [801/801 units], [G02 😈] [4485/4485 units]
[G02 😈] ⚔ (107640 💫) (-84 units) [G02 😇], [G02 😇] ⚔ (22625 💫) (-4 units) [G01 😈], [G01 😇] ⚔ (153238 🔥) (-51 units) [G02 😈], [G01 😈] ⚔ (184904 👊) (-34 units) [G01 😇], 

Round 2
[G02 😇] [905/989 units], [G01 😈] [797/801 units], [G02 😈] [4434/4485 units]
[G02 😇] ⚔ (22625 💫) (-4 units) [G01 😈], [G01 😈] ⚔ (183976 👊) (-144 units) [G02 😇], 

Round 3
[G02 😇] [761/989 units], [G01 😈] [793/801 units], [G02 😈] [4434/4485 units]
[G02 😇] ⚔ (19025 💫) (-4 units) [G01 😈], [G01 😈] ⚔ (183048 👊) (-143 units) [G02 😇], 

Round 4
[G02 😇] [618/989 units], [G0